## 2021: Week 36 - What's Trendy?

For this week's challenge, I wanted to use Google Trends to take a look back over the past couple of years and see what people were searching for. In particular, are these categories still as popular now as they were in peak lockdown? How does the experience vary around the world? We'll be looking at:

- Pet adoption (who didn't want a furry work from home buddy?!)
- Online streamer (can one make money from playing video games?)
- Staycations (everyone's favourite word, right?)

### Input
There are 2 inputs this week:
1. Timeline - indexes for how popular the term is
![img](https://lh3.googleusercontent.com/-lVnuEqzTNtY/YS-ZW0sI3xI/AAAAAAAAA7Y/30m_U5C0SuYvhfe0Mssodb3bW7jlqQvDgCLcBGAsYHQ/w400-h126/image.png)

2. Country Breakdown - the percentage of these terms popularity in each country
![img](https://lh3.googleusercontent.com/-bZ4c-74Ebz8/YS-ZfQbLv_I/AAAAAAAAA7c/r-lB8iGWrAoWjQVe_HXx2nJ7bfycgyFzwCLcBGAsYHQ/w400-h85/image.png)

### Requirement
- Input the data
- Calculate the overall average index for each search term
- Work out the earliest peak for each of these search terms
- For each year (1st September - 31st August), calculate the average index
- Classify each search term as either a Lockdown Fad or Still Trendy based on whether the average index has increased or decreased since last year
- Filter the countries so that only those with values for each search term remain
- For each search term, work out which country has the highest percentage
- Bring everything together into one dataset
- Output the data

### Output
![img](https://lh3.googleusercontent.com/-NwnYxJvHKCQ/YS-bPi52xnI/AAAAAAAAA7k/oIYuwkhV79kyMnmelYviOZygovSXnSmagCLcBGAsYHQ/w400-h58/image.png)

- 7 fields
    - Search Term
    - Status
    - 2020/21 avg index
    - Avg index
    - Index Peak
    - First Peak
    - Country with highest percentage
- 3 rows (4 including headers)

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Input the data

In [15]:
data = pd.read_excel("./data/Trend Input.xlsx", sheet_name=[0, 1])
timeline = data[0].copy()
country_breakdown = data[1].copy()

In [16]:
timeline

,Category: All categories,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Week,Pet adoption: (Worldwide),Online streamer: (Worldwide),Staycation: (Worldwide)
2,2016-09-04 00:00:00,69,11,6
3,2016-09-11 00:00:00,70,10,4
4,2016-09-18 00:00:00,64,17,3
...,...,...,...,...
258,2021-08-01 00:00:00,52,46,42
259,2021-08-08 00:00:00,56,48,42
260,2021-08-15 00:00:00,57,48,42
261,2021-08-22 00:00:00,61,48,37


In [17]:
timeline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Category: All categories  262 non-null    object
 1   Unnamed: 1                262 non-null    object
 2   Unnamed: 2                262 non-null    object
 3   Unnamed: 3                262 non-null    object
dtypes: object(4)
memory usage: 8.3+ KB


In [18]:
country_breakdown

,Category: All categories,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Country,Pet adoption: (01/09/2016 - 01/09/2021),Online streamer: (01/09/2016 - 01/09/2021),Staycation: (01/09/2016 - 01/09/2021)
2,Hong Kong,0.03,0.05,0.92
3,South Korea,NaN,NaN,NaN
4,Guernsey,NaN,NaN,1
...,...,...,...,...
247,Tuvalu,NaN,NaN,NaN
248,US Outlying Islands,NaN,NaN,NaN
249,Vatican City,NaN,NaN,NaN
250,Wallis & Futuna,NaN,NaN,NaN


In [19]:
country_breakdown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Category: All categories  251 non-null    object
 1   Unnamed: 1                21 non-null     object
 2   Unnamed: 2                22 non-null     object
 3   Unnamed: 3                24 non-null     object
dtypes: object(4)
memory usage: 8.0+ KB


### Calculate the overall average index for each search term

In [20]:
timeline.columns = ["Week", "Pet_adoption", "Online_streamer", "Staycation"]
timeline = timeline.drop([0, 1], axis=0)
timeline = timeline.reset_index(drop=True)

In [21]:
timeline

,Week,Pet_adoption,Online_streamer,Staycation
0,2016-09-04 00:00:00,69,11,6
1,2016-09-11 00:00:00,70,10,4
2,2016-09-18 00:00:00,64,17,3
3,2016-09-25 00:00:00,64,13,3
4,2016-10-02 00:00:00,63,12,5
...,...,...,...,...
256,2021-08-01 00:00:00,52,46,42
257,2021-08-08 00:00:00,56,48,42
258,2021-08-15 00:00:00,57,48,42
259,2021-08-22 00:00:00,61,48,37
